In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [2]:
url = "https://www.cuny.edu/academics/academic-calendars/fall-2021/"
response = requests.get(url)

In [3]:
response

<Response [200]>

In [12]:
soup = BeautifulSoup(response.text, 'html.parser')

# Inspect the HTML structure (optional)
print(soup.prettify())

# Locate the table containing the data (assuming it's in <tbody>)
calendar_table = soup.find('tbody')

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html lang="en-gb" xml:lang="en-gb" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   CUNY.edu Temporarily Unavailable
  </title>
  <link href="maintenance/main.css" rel="stylesheet" type="text/css"/>
  <script src="maintenance/jquery-1.3.2.js" type="text/javascript">
  </script>
  <script src="maintenance/main-wrap.js" type="text/javascript">
  </script>
 </head>
 <body>
  <div id="shadowbox">
   <div class="container">
    <div id="atticbar">
    </div>
    <div class="header">
     <table class="layout-960-3">
      <tbody>
       <tr>
        <td class="left-270" style="padding-top: 20px;">
         <a href="http://www.cuny.edu" title="CUNY Home Page">
          <img alt="CUNY Logo" src="maintenance/cuny-logo.gif"/>
         </a>
        </td>
        <td class="mid-380" style="vertical-align:middle; padding-top: 20px;">
        </td>
        <td class="right-270">

In [9]:
def date_handler(s):
    s = s.strip()

    if not s or s == "2021":
        return None

    if " - " in s:
        # Handle stuff like "December 15 - 21"
        start_part, end_day = s.split(" - ")
        start_date_obj = datetime.strptime(f"{start_part} 2021", "%B %d %Y")
        return f"{start_date_obj.year}-{start_date_obj.month:02d}-({start_date_obj.day} - {end_day})"
    else:
        if "2022" in s or "," in s:
            single_date_obj = datetime.strptime(s, "%B %d, %Y")
        else:
            single_date_obj = datetime.strptime(f"{s} 2021", "%B %d %Y")
        return single_date_obj.strftime("%Y-%m-%d")

In [10]:
calendar = []

In [13]:
for row in calendar_table.find_all('tr'):
    columns = row.find_all('td')

    if len(columns) == 3:
        date_info = columns[0].get_text(strip=True)
        weekday_info = columns[1].get_text(strip=True)
        description_info = columns[2].get_text(strip=True)

        # Convert the date using the custom function
        formatted_date = date_handler(date_info)

        # Append the extracted data to the list
        calendar_data.append({
            "date": formatted_date,
            "dow": weekday_info,
            "text": description_info
        })

ValueError: time data ' 2021' does not match format '%B %d %Y'